<a href="https://colab.research.google.com/github/erobinson/nightscout-ml/blob/main/nightscout_ml_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to nightscout-ml


In [8]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive/', force_remount=True)

data_folder = '/content/drive/My Drive/nightscout-ml/'
data_xlsx_file = data_folder+'/data.xlsx'
path = Path(data_xlsx_file)
if not path.is_file():
  raise Exception("Data file not found in location: "+data_xlsx_file)
else:
  print("👍 Found data file at location: "+data_xlsx_file)

Mounted at /content/drive/
👍 Found data file at location: /content/drive/My Drive/nightscout-ml/mt/data.xlsx


In [9]:
from datetime import datetime

date_format_str = '%m/%d/%y %I:%M%p'

def time_to_str(date):
    date_str = datetime.strftime(date, date_format_str)
    
    # drop leading 0 on month
    date_str = date_str[1:] if date_str.startswith('0') else date_str
    
    date_str = date_str.replace('/0', '/')
    return date_str

def str_to_time(date_str):
    return datetime.strptime(date_str, date_format_str)

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU
import time
from tensorboard.plugins.hparams import api as hp
import os
import shutil

pd.set_option('display.width', 3000)
now = datetime.now()
date_str = "{}-{}-{}_{}-{}".format(now.year, now.month, now.day, now.hour, now.minute)
time_ranges = ['0to60minAgo', '60to120minAgo', '120to180minAgo', '180to240minAgo']
log_folder = 'logs'

hour_breakdowns = ["hour0_2","hour3_5","hour6_8","hour9_11","hour12_14","hour15_17","hour18_20","hour21_23"]
accellerating = ["accelerating_up","deccelerating_up","accelerating_down","deccelerating_down","stable"]
recent_steps = ["recentSteps5Minutes","recentSteps10Minutes","recentSteps15Minutes","recentSteps30Minutes","recentSteps60Minutes"]
sleep_seated = ["sleep","sedentary"]
tdd = ["tdd7Days","tddDaily","tdd24Hrs"]
tddPerHour = ["tdd7DaysPerHour","tdd2DaysPerHour","tddDailyPerHour","tdd24HrsPerHour"]
base_cols = ["hourOfDay","weekend",
                "bg","targetBg","iob","cob","lastCarbAgeMin","futureCarbs",
                "delta","shortAvgDelta","longAvgDelta"]
cob_delta = ['cobDelta']
gi_tags = ['giFast', 'giMedium', 'giSlow']
prior_bgs_to_add = 36

col_map = {
    'best_cols': base_cols+tddPerHour+recent_steps,
    # 'best_cols_cob_delta': base_cols+tddPerHour+cob_delta+recent_steps+cob_delta,
    # 'best_cols_plus_glycemic_index': base_cols+tddPerHour+recent_steps+gi_tags,
    # 'best_cols_plus_fast_med_slow': base_cols+tddPerHour+recent_steps+['fastCarbs','mediumCarbs','slowCarbs']
    # 'best_cols_plus_prior_bgs_and_smbs': base_cols+tddPerHour
    # 'best_cols_w_accell': base_cols+tddPerHour+cob_delta+recent_steps+accellerating,
    
    ### meal tags didn't improve model accuracy sadly :(
    # 'best_cols_plus_tags': base_cols+tddPerHour+cob_delta+recent_steps

    #'base': base_cols, # comment this line
    # 'base_cobDelta': base_cols+cob_delta, # helps a little
    # 'base_tddPerhour': base_cols+tddPerHour, # helpful
    # 'base_recentSteps': base_cols+recent_steps, # helpful
    # 'base_accellerating': base_cols+accellerating, # helpful, but could probably refine
    
    # 'base_tdd': base_cols+tdd, # helps, but tddPerHour values are more effective
    # 'base_tdd_tddPerHour': base_cols+tdd+tddPerHour, # not more helpful than tddPerHour
    # 'base_recentSteps_sleepSeated': base_cols+recent_steps+sleep_seated, # no major diff, steps is fine
    # 'base_sleepSeated': base_cols+sleep_seated, # no major diff thus drop
    # 'base_hour_breakdowns': base_cols+hour_breakdowns # not helpful thus drop
}

# Define model hyper parameters such as input values and nural network dimensions
HP_COLS = hp.HParam('cols', hp.Discrete(list(col_map.keys())))

HP_NUM_NODES_L1 = hp.HParam('num_nodes_l1', hp.Discrete([5])) 
HP_NUM_NODES_L2 = hp.HParam('num_nodes_l2', hp.Discrete([0]))
HP_NUM_NODES_L3 = hp.HParam('num_nodes_l3', hp.Discrete([0]))
HP_NUM_NODES_L4 = hp.HParam('num_nodes_l4', hp.Discrete([0]))
HP_NUM_EPOCHS = hp.HParam('num_epochs', hp.Discrete([2]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([.1]))
HP_LAST_ACTIVATION = hp.HParam('last_activation', hp.Discrete(['prelu']))
METRIC_LOSS = 'loss'

def build_model(run_dir, hparams, train_features, train_labels, test_features, test_labels):
    loss = 1
    model = ''
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        model, loss = train_test_model(hparams, train_features, train_labels, test_features, test_labels)
        tf.summary.scalar(METRIC_LOSS, loss, step=1)
    return model, loss

def train_test_model(hparams, train_features, train_labels, test_features, test_labels):
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))

    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(train_features.shape[1],)))
    model.add(normalizer)
    if hparams[HP_NUM_NODES_L1] > 0:
        model.add(layers.Dense(units=hparams[HP_NUM_NODES_L1], activation="relu"))
    if hparams[HP_NUM_NODES_L2] > 0:
        model.add(layers.Dense(units=hparams[HP_NUM_NODES_L2], activation="relu"))
    if hparams[HP_NUM_NODES_L3] > 0 and hparams[HP_NUM_NODES_L2] > 0:
        model.add(layers.Dense(units=hparams[HP_NUM_NODES_L3], activation="relu"))
    if hparams[HP_NUM_NODES_L4] > 0 and hparams[HP_NUM_NODES_L3] > 0 and hparams[HP_NUM_NODES_L2] > 0:
        model.add(layers.Dense(units=hparams[HP_NUM_NODES_L4], activation="relu"))

    if hparams[HP_LAST_ACTIVATION] == 'prelu':
        prelu = PReLU()
        model.add(layers.Dense(units=1, activation=prelu))
    else:
        model.add(layers.Dense(units=1, activation=hparams[HP_LAST_ACTIVATION]))

    model.compile(
        # optimizer=tf.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE]),
        optimizer='adam',
        loss='mean_absolute_error')

    model.fit(
        train_features,
        train_labels,
        epochs=hparams[HP_NUM_EPOCHS],
        # Suppress logging.
        verbose=1,
        # Calculate validation results on 20% of the training data.
        validation_split = 0.2)
    
    loss = model.evaluate(test_features, test_labels)
    print(train_features.head(10))
    print(train_labels.head(10))
    print(test_features.head(10))
    print(test_labels.head(10))
    print(f"Trained model with {len(model.layers)} layers and loss of {loss}")
    return model, loss

def add_glycemic_index_cols(df):
    # load vocabulary map from csv
    df_gi = pd.read_csv('glycemic_index.csv')
    for index, row in df_gi.iterrows():
        tag = row['tag']
        gi = row['index']
        
        # if 0-60 min contains fast acting carb word, then set fast GI to 1
        if gi == 'fast':
            df['giFast'] = df['tags0to60minAgo'].str.contains(tag)
            df['giFast'] = df['giFast'].astype(int)
        
        # if 0-60 or 60-120 contains medium acting carb word, then set medium GI col to 1
        if gi == 'medium':
            df['giMedium'] = df['tags0to60minAgo'].str.contains(tag)
            df['giMedium'] = df['tags60to120minAgo'].str.contains(tag)
            df['giMedium'] = df['giMedium'].astype(int)
        
        # if tags contain slow acting word, set slow GI column to 1
        if gi == 'slow':
            df['giSlow'] = df['tags60to120minAgo'].str.contains(tag)
            df['giSlow'] = df['tags120to180minAgo'].str.contains(tag)
            df['giSlow'] = df['tags180to240minAgo'].str.contains(tag)
            df['giSlow'] = df['giSlow'].astype(int)
        
    return df

def convert_tags_to_cols(df):
    vocabulary = get_vocabulary_from_tags(df)
    
    # fill in NaN values with empty strings to support setting tags
    for range in time_ranges:
        df['tags'+range] = df['tags'+range].fillna('')

    for word in vocabulary:
        for range in time_ranges:
            col_name = word+'_'+range
            df[col_name] = df['tags'+range].str.contains(word)
            df[col_name] = df[col_name].astype(int)
    return df, vocabulary

def get_vocabulary_from_tags(df):
    # get list of unique words
    words = list(df['tags0to60minAgo'].str.split(' ', expand=True).stack().unique())
    stop_words = ['for', 'a', 'an', 'and', 'of', 'with', '']
    
    # vocaublary = [word.lower() for word in vocaublary]
    vocabulary = []
    for word in words:
        word = word.lower()
        word = word[:-2] if word.endswith("es") else word
        word = word[:-1] if word.endswith("s") else word
        if word not in stop_words and word not in vocabulary:
            vocabulary.append(word)
    # strip commas, strip stop words (and, a) - android strips a/an/and
    # stem words if need be
    return vocabulary

def get_basic_model_desc(model):
    model_info = "\n\n------------\n"
    model_info += f"Model {date_str}\n"
    model_info += f"{len(model.layers)} Layers:\n"
    for i in range (len(model.layers)):
        layer = model.layers[i]
        layer_info = f"    - Layer {i}  - {layer.name}"
        layer_info += f" ({layer.units})" if 'dense' in layer.name else ""
        layer_info += f" ({layer.rate})" if 'dropout' in layer.name else ""
        model_info += layer_info + "\n"
    return model_info

def save_model_info_v2(model, cols, best_loss, num_epochs, data_row_count, training_time, best_last_activation, best_learning_rate, vocabulary):
    model_info = get_basic_model_desc(model)

    model_info += f"Model Loss & Accuracy: {str(round(best_loss, 5))}\n"
    model_info += f"Number of Epochs: {num_epochs} \n"
    model_info += f"Columns ({len(cols)}): {cols} - {col_map[cols]} \n"
    model_info += f"Vocabulary ({len(vocabulary)}) {vocabulary}\n"
    model_info += f"Training Data Size: {data_row_count} \n"
    model_info += f"Learning Rate: {best_learning_rate} \n"
    model_info += f"Activation: {best_last_activation} \n" if best_last_activation is not None else "Activation: None\n"
    # model_info += basic_predictions(model) + "\n"
    model_info += f"Took {time.strftime('%H:%M:%S', time.gmtime(training_time))} to train\n"
    model_info += "NOTES: \n"
    open(data_folder+'/models/tf_model_results.txt', "a").write(model_info)



def basic_predictions(model):
    if len(current_cols) != 38:
        return f"ERROR: incorrect number of columns ({len(current_cols)})"

    low = basic_predict(model,50,0.0,0.0,0)
    low_w_iob = basic_predict(model,50,1.0,0.0,0)
    normal_w_iob = basic_predict(model,100,1.0,0.0,0)
    normal_wo_iob = basic_predict(model,100,0.0,0.0,0)
    high_bg = basic_predict(model,200,0.0,0.0,0)
    high_cob = basic_predict(model,100,1.0,30.0,0)
    high_both = basic_predict(model,200,1.0,30.0,0)
    line =  f"    low: {low}    low_w_iob: {low_w_iob}    normal_w_iob: {normal_w_iob}    normal_wo_iob: {normal_wo_iob}\n"
    line += f"    high_bg: {high_bg}    high_cob: {high_cob}    high_both: {high_both}    high_both_and_rising {basic_predict(model, 200, 1, 60, 10)}\n"
    line += f"    low_rising  : {basic_predict(model, 70, 0, 20, 10)}    normal_rising  : {basic_predict(model, 100, 0, 20, 10)}    high_rising  : {basic_predict(model, 180, 0, 20, 10)}\n"
    line += f"    low_dropping: {basic_predict(model, 70, 2, 0, -7)}    normal_dropping: {basic_predict(model, 100, 2, 0, -7)}    high_dropping: {basic_predict(model, 180, 2, 0, -7)}\n"
    return line
    
def basic_predict(model, bg, iob, cob, delta):
    last_cob_min = 0 if cob == 0 else 5
    accelerating_up = 1 if delta > 3 else 0
    deccelerating_down = 1 if delta < -3 else 0
    stable= 1 if delta > -3 and delta < 3 else 0
    prediction = model.predict([[11,1,0,0, 0,0,0, 0,0,0, 
                    bg,100,iob,cob,last_cob_min,0,delta,delta,delta,
                    accelerating_up, 0, deccelerating_down, 0, stable, 
                    33,1, 33,1, 33,1,
                    0,0,0, 0,0,
                    0,1]])
                    # "hourOfDay","hour0_2","hour3_5","hour6_8", "hour9_11","hour12_14","hour15_17", "hour18_20","hour21_23","weekend",
                    # "bg","targetBg","iob","cob","lastCarbAgeMin","futureCarbs","delta","shortAvgDelta","longAvgDelta",
                    # "accelerating_up","deccelerating_up","accelerating_down","deccelerating_down","stable",
                    # "tdd7Days","tddDaily","tddPerHour","tdd24Hrs",
                    # "tdd7Days","tdd7DaysPerHour", "tddDaily","tddDailyPerHour", "tdd24Hrs","tdd24HrsPerHour",
                    # "recentSteps5Minutes","recentSteps10Minutes","recentSteps15Minutes", "recentSteps30Minutes","recentSteps60Minutes",
                    # "sleep","sedintary",
                    
    return str(round(prediction[0][0], 3))

def clear_log_folder():
    if os.path.exists(log_folder):
        shutil.rmtree(log_folder)
    os.mkdir(log_folder)
    if not os.path.exists(data_folder+'/models'):
        os.mkdir(data_folder+'/models')



def save_model(model):
    # https://www.tensorflow.org/tutorials/keras/save_and_load#savedmodel_format
    model.save(data_folder+'/models/backup/tf_model_'+date_str)

    # https://medium.com/analytics-vidhya/running-ml-models-in-android-using-tensorflow-lite-e549209287f0
    converter = tf.lite.TFLiteConverter.from_keras_model(model=model)
    lite_model = converter.convert()
    open(data_folder+'/models/backup/tf_model_'+date_str+'.tflite', "wb").write(lite_model)
    open(data_folder+'/models/model.tflite', "wb").write(lite_model)


def compare_two_models(model_1_date, model_2_date, row_dates):
    m1 = tf.keras.models.load_model(f'{data_folder}/models/backup/tf_model_{model_1_date}')
    m2 = tf.keras.models.load_model(f'{data_folder}/models/backup/tf_model_{model_2_date}')

    df = pd.read_excel(data_xlsx_file,'training_data')
    
    # row_1 = row.copy()
    # row_1.pop('tdd7DaysPerHour')
    # row_1.pop('tdd24HrsPerHour')

    eval_features = df.sample(frac=0.3, random_state=0)
    eval_features = eval_features[current_cols]
    eval_features_1 = eval_features.copy()
    # eval_features_1.pop('tdd7DaysPerHour')
    # eval_features_1.pop('tdd24HrsPerHour')
    eval_features_1.pop('smbToGive')
    eval_labels = eval_features.pop('smbToGive')

    m1_eval = str(round(m1.evaluate(eval_features_1, eval_labels)[0], 6))
    m2_eval = str(round(m2.evaluate(eval_features, eval_labels)[0], 6))
    eval_diff = str(round((float(m2_eval) - float(m1_eval)), 5))

    # m1_predict = str(round(m1.predict([row_1])[0][0],3))
    # m2_predict = str(round(m2.predict([row])[0][0],3))

    model_info = " ---- Model Comparison ----\n"
    model_info += f" ---- MODELS: model 1: {model_1_date} - model 2: {model_2_date}\n"
    model_info += f" ---- EVAL LOSS:   model 1: {m1_eval} - model 2: {m2_eval} => "
    model_info += f"{(eval_diff)} getting better \n\n" if float(eval_diff) < 0 else f"{(eval_diff)} !!! BAD !!! \n\n"
    for row_date in row_dates:
        row = df.loc[df['dateStr'] == row_date]
        row = row[current_cols]
        smb_to_give = row['smbToGive'].values[0]
        row.pop('smbToGive')
        m1_predict = str(round(m1.predict([row])[0][0],3))
        m2_predict = str(round(m2.predict([row])[0][0],3))
        model_info += f" ------ PREDICTIONS: {row_date} "
        model_info += f"({smb_to_give}u, {row['bg'].values[0]}mg/dL, delta: {row['delta'].values[0]}, shortAvgDelta: {row['shortAvgDelta'].values[0]})"
        model_info += f" - model 1: {m1_predict} - model 2: {m2_predict}\n"
    
    open(data_folder+'/models/tf_model_results.txt', "a").write(model_info)



def build_tf_regression():
    start = time.time()
    df = pd.read_excel(data_xlsx_file,'training_data')
    # df = df[100:9566]
    clear_log_folder()

    with tf.summary.create_file_writer(f'{log_folder}/hparam_tuning').as_default():
        hp.hparams_config(
            hparams=[HP_COLS, HP_NUM_NODES_L1, HP_NUM_NODES_L2, HP_NUM_NODES_L3, HP_NUM_NODES_L4, 
            HP_NUM_EPOCHS, HP_LEARNING_RATE, HP_LAST_ACTIVATION],
            metrics=[hp.Metric(METRIC_LOSS, display_name='loss')],
        )

    if 'best_cols_plus_glycemic_index' in col_map.keys() or 'best_cols_plus_tags' in col_map.keys():
        df, vocabulary = convert_tags_to_cols(df)
        if 'best_cols_plus_glycemic_index' in col_map.keys():
            df = add_glycemic_index_cols(df)
    else:
        vocabulary = ['not using tags']
    train_dataset = df.sample(frac=0.8, random_state=0)
    test_dataset = df.drop(train_dataset.index)

    train_features = train_dataset.copy()
    test_features = test_dataset.copy()

    train_labels = train_features.pop('smbToGive')
    test_labels = test_features.pop('smbToGive')

    best_loss = 1
    best_model = 1
    best_epochs = 0
    best_last_activation = ''
    best_learning_rate = .1
    best_cols = ''

    
    
    session_num = 0

    for cols_name in HP_COLS.domain.values:
        columns = col_map[cols_name]
        if 'plus_tags' in cols_name:
            for time_range in time_ranges:
                for word in vocabulary:
                    columns.append(f"{word}_{time_range}")
        if 'prior_bgs_and_smbs' in cols_name:
            for offset in range(0, prior_bgs_to_add):
                offset_in_5_min = (offset+1) * 5
                columns.append(f"{offset_in_5_min}_min_prior_bg")
                columns.append(f"{offset_in_5_min}_min_prior_smb")

        iteration_train_features = train_features[columns]
        iteration_test_features = test_features[columns]
        for num_nodes_l1 in HP_NUM_NODES_L1.domain.values:
            for num_nodes_l2 in HP_NUM_NODES_L2.domain.values:
                for num_nodes_l3 in HP_NUM_NODES_L3.domain.values:
                    for num_nodes_l4 in HP_NUM_NODES_L4.domain.values:
                        for num_epochs in HP_NUM_EPOCHS.domain.values:
                            for learn_rate in HP_LEARNING_RATE.domain.values:
                                for last_activation in HP_LAST_ACTIVATION.domain.values:
                                    hparams = {
                                        HP_COLS: cols_name,
                                        HP_NUM_NODES_L1: num_nodes_l1,
                                        HP_NUM_NODES_L2: num_nodes_l2,
                                        HP_NUM_NODES_L3: num_nodes_l3,
                                        HP_NUM_NODES_L4: num_nodes_l4,
                                        HP_NUM_EPOCHS: num_epochs,
                                        HP_LEARNING_RATE: learn_rate,
                                        HP_LAST_ACTIVATION: last_activation
                                    }
                                    run_name = f"run-{cols_name}-{session_num}"
                                    print(f"--- Starting trail {run_name}")
                                    print({h.name: hparams[h] for h in hparams})
                                    model, loss = build_model('logs/hparam_tuning/' + run_name, hparams, iteration_train_features, train_labels, iteration_test_features, test_labels)
                                    session_num += 1
                                    if loss < best_loss:
                                        best_loss = loss
                                        best_model = model
                                        best_epochs = num_epochs
                                        best_learning_rate = learn_rate
                                        best_last_activation = last_activation
                                        best_cols = cols_name
    
    training_time = time.time() - start
    
    save_model_info_v2(best_model, best_cols, best_loss, best_epochs, len(df), training_time, best_last_activation, best_learning_rate, vocabulary)

    save_model(best_model)
    
    return date_str

build_tf_regression()

--- Starting trail run-best_cols-0
{'cols': 'best_cols', 'num_nodes_l1': 5, 'num_nodes_l2': 0, 'num_nodes_l3': 0, 'num_nodes_l4': 0, 'num_epochs': 2, 'learning_rate': 0.1, 'last_activation': 'prelu'}
Epoch 1/2
71/71 [==============================] - 1s 7ms/step - loss: 0.1056 - val_loss: 0.0757
Epoch 2/2
23/23 [==============================] - 0s 2ms/step - loss: 0.0851
      hourOfDay  weekend     bg  targetBg       iob  cob  lastCarbAgeMin  futureCarbs  delta  shortAvgDelta  longAvgDelta  tdd7DaysPerHour  tdd2DaysPerHour  tddDailyPerHour  tdd24HrsPerHour  recentSteps5Minutes  recentSteps10Minutes  recentSteps15Minutes  recentSteps30Minutes  recentSteps60Minutes
2606          6        0   98.0      90.0  0.035672  0.0            1440          0.0    1.0           0.33          0.69         1.346037         1.353941         1.319368         1.131111                    0                     0                     0                     0                     0
2010          0        0  1

'2023-4-3_14-0'

In [23]:
# Testing Model
import tensorflow as tf
import numpy as np
file_path = f'{data_folder}/model-2.tflite'
if not Path(file_path).is_file():
  raise Exception("Data file not found in location: "+data_xlsx_file)

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=file_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()
print(output_details)

def run_prediction(input_data):
  # https://stackoverflow.com/questions/50443411/how-to-load-a-tflite-model-in-script
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()
  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  return output_data


# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print("prediction with random data: "+str(run_prediction(input_data)))

# input columns: 
#   'hourOfDay', 'weekend', 'bg', 'targetBg', 'iob', 'cob', 'lastCarbAgeMin', 'futureCarbs', 
#   'delta', 'shortAvgDelta', 'longAvgDelta', 
#   'tdd7DaysPerHour', 'tdd2DaysPerHour', 'tddDailyPerHour', 'tdd24HrsPerHour', 
#   'recentSteps5Minutes', 'recentSteps10Minutes', 'recentSteps15Minutes', 'recentSteps30Minutes', 'recentSteps60Minutes'
input_data = [[0, 1, 200, 100, 1, 60, 5, 0,
               10, 10, 10,
               1.5, 1.5, 1.5, 1.5,
               10, 10, 10, 10, 10]]
for i in range(len(input_data[0])):
  # model requires float 32 numbers
  input_data[0][i] = np.float32(input_data[0][i])

print("prediction with input data: "+str(run_prediction(input_data)))

[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([ 1, 25], dtype=int32), 'shape_signature': array([-1, 25], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 12, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
prediction with random data: [[-137.01595]]


ValueError: ignored